importing colab to read the google drive files

In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


importing all the necessary libraries

In [123]:
import pandas as pd
import re
import os
import nltk
from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import PorterStemmer
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

code to get the list of all files in the folder

In [3]:
list1=os.listdir("/content/gdrive/MyDrive/Humor,Hist,Media,Food");

code to read the content of the file

In [5]:
file=open("/content/gdrive/MyDrive/Humor,Hist,Media,Food/"+"awespinh.sal","r",errors="ignore",encoding ="utf-8")
txt=file.read()

contains the code to preprocess the raw query text

In [118]:
def my_stemming(my_Text):
  ps = PorterStemmer()
  idx = 0 
  for word in my_Text:
      my_Text[idx] = ps.stem(word)
      idx = idx + 1

In [120]:
def process(txt):

  #punctuation removal
  punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~=+'''
  for ele in txt:
    if ele in punc:
        txt = txt.replace(ele, " ")
  
  #lower the text
  txt_lower=txt.lower()

  #remove stopwords
  txt_stopwords=remove_stopwords(txt_lower)

  #tokenization
  txt_tokenize=nltk.word_tokenize(txt_stopwords)

  #remove blank spaces
  for word in txt_tokenize:
    word=word.strip() 

  #stemming
    my_stemming(txt_tokenize)

  #remove duplicates 
  #txt_tokenize=list(set(txt_tokenize))

  return txt_tokenize 

code to build positional index datastructure using python dictonaries

In [ ]:
dict_index={};
dict_positional_index={};
dict_frequency={};
index=0
for doc in list1:
  file=open("/content/gdrive/MyDrive/Humor,Hist,Media,Food/"+doc,"r",errors="ignore",encoding ="utf-8")
  dict_index[index]=doc
  txt=file.read()
  tokens=process(txt)
  pos=0
  for token in tokens:
    if token in dict_positional_index:
      if index in dict_positional_index[token]:
        dict_positional_index[token][index].append(pos)
      else:
        dict_positional_index[token][index]=[]
        dict_positional_index[token][index].append(pos)
      dict_frequency[token]=dict_frequency[token]+1
    else:
      dict_positional_index[token]={}
      dict_positional_index[token][index]=[]
      dict_positional_index[token][index].append(pos)
      dict_frequency[token]=1
    pos+=1
  index+=1


saving the positional index datastructure to use it again when requires

In [14]:
filehandler = open("dict_postional_index_1.txt", 'wt',errors="ignore",encoding ="utf-8")
data = str(dict_positional_index)
filehandler.write(data) 

10543215

saving the datastructure - containing no of documents in which a particular word present

In [15]:
filehandler = open("dict_frequency_1.txt", 'wt',errors="ignore",encoding ="utf-8")
data = str(dict_frequency)
filehandler.write(data) 

1334866

saving the data structure - containing documents id's of each document

In [17]:
filehandler = open("dict_index_1.txt", 'wt',errors="ignore",encoding ="utf-8")
data = str(dict_index)
filehandler.write(data) 

22204

Reading the positional index datastructure to use it again
& using AST library to get the syntax of data structure

In [ ]:
import ast
file_result = open("/content/dict_postional_index_1.txt", 'r',errors="ignore",encoding ="utf-8")
result1=file_result.read()
dic=ast.literal_eval(res)


Reading the documents -index datastructure using AST library(ABSTRACT SYNTAX TREE)

In [96]:
file_second = open("/content/dict_index_1.txt", 'r',errors="ignore",encoding ="utf-8")
result=file_second.read()
dic2=ast.literal_eval(result)

function to return the documents in which the phrasal query present
input-- query
output -- documents id

In [104]:
def getDoc(str):
  str=process(str)
  if len(str)==0:
    return [{}]
  elif len(str)==1:
    return dic[str[0]]
  else:
    listing_curr=list(dic[str[0]].keys())
    for next in range(1,len(str)):
      curr=next-1
      global_documents=[]
      listing_next=list(dic[str[next]].keys())
      i=0
      j=0
      while(i<len(listing_curr) and j<len(listing_next)):
        documents={}
        if listing_curr[i]==listing_next[j] :
          #if(curr==0):
          list_A=dic[str[curr]][listing_curr[i]]
          #else:
            #list_A=global[listing_curr[i]]
          list_B=dic[str[next]][listing_next[j]]
          m=0
          n=0
          position=[]
          while(m<len(list_A) and n<len(list_B)):
            if(list_A[m]+1==list_B[n]):
              position.append(list_B[n])
              m+=1
              n+=1
            elif list_A[m]<list_B[n]:
              m+=1
            else:
              n+=1
          if(len(position)>0):
            documents[listing_next[j]]=position
            global_documents.append(documents)
          i+=1
          j+=1
        elif listing_curr[i]<listing_next[j] :
          i+=1
        else:
          j+=1
      if(len(documents)>0):
        listing_curr=list(documents.keys())
      else:
        break
    return global_documents


function to get the documents name from the documents id

In [97]:
def getDocName(global_documents):
  lis=[]
  for i in range(0,len(global_documents)):
    lis.append(list(global_documents[i].keys())[0])
  doc_name=[]
  for j in lis:
    doc_name.append(dic2[j])
  return doc_name


here we take query as user input and print -
1) Number of documents in which the query presents 
2) Name of the documents in which the query presents

In [ ]:
str=input("enter query to search")
docs=getDoc(str)
print(len(docs))
names=getDocName(docs)
print(names)